# Simulated Data for FTS Analysis Code assuming Monochromatic Source

#### Lindsay Ng Lowry
#### 20181023

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pkl

In [41]:
cd /Users/lindsay/Documents/UCSD/POLARBEAR_Research/FourierTransformSpectrometer/Code/LabTesting/Mapfiles

/Users/lindsay/Documents/UCSD/POLARBEAR_Research/FourierTransformSpectrometer/Code/LabTesting/MapFiles


In [42]:
# Load the mapfile and convert positions from encoder counts to m - should be max range/resolution
mapfile = 'FTSScanPoints_DEFAULT.txt'

positions = np.loadtxt(mapfile)
print "Positions (encoder counts): ", positions

# Convert from encoder counts to inches
steps_per_inch = 125000.0
positions_inches = positions/steps_per_inch
print "Positions (inches): ", positions_inches

# Convert from inches to m
positions_m = positions_inches*(1./39.3701)
print "Positions (m): ", positions_m

Positions (encoder counts):  [-738000. -737262. -736524. ...,  736524.  737262.  738000.]
Positions (inches):  [-5.904    -5.898096 -5.892192 ...,  5.892192  5.898096  5.904   ]
Positions (m):  [-0.14996152 -0.14981156 -0.1496616  ...,  0.1496616   0.14981156
  0.14996152]


In [43]:
# Convert from position to path length difference (multiply by 2)
path_diffs = positions_m*2.0
print path_diffs

[-0.29992304 -0.29962312 -0.29932319 ...,  0.29932319  0.29962312
  0.29992304]


In [44]:
# Function to calculated the intensity for a given path length difference and wavelength assuming I_0 = 1
    # Eq 8 in https://pubs.acs.org/doi/pdf/10.1021/ed2002253
def calc_intensity(wavelength, path_diff):
    return 0.5*(1 + np.cos((2*np.pi*path_diff)/(wavelength)))

In [45]:
# Function to add a linear trend to the data
def linear_trend(x,slope):
    return slope*x

In [46]:
# Make an array containing the calculated ideal intensities for each mirror position
# Add a linear trend on top of the data if desired
wavelengths = np.arange(0.0005,0.0030,0.0001)

interf_data = []
for pd in path_diffs:
    signal = 0
    for w in wavelengths:
        signal += calc_intensity(w,pd)
    
    linear_addition = linear_trend(pd,15)
    signal += linear_addition
    interf_data.append(signal)
    
interf_data = np.array(interf_data)
print len(interf_data)
print len(interf_data) == len(positions)



2001
True


In [47]:
# Add the position and signal data to a dictionary mimicking the data output from Nate's cryostat
data = {}
for p, signal in zip(positions, interf_data):
    data[p] = signal

In [48]:
# Plot the interferogram
plt.figure()
plt.plot(positions_inches,interf_data)
plt.show()

In [49]:
cd /Users/lindsay/Documents/UCSD/POLARBEAR_Research/FourierTransformSpectrometer/Data

/Users/lindsay/Documents/UCSD/POLARBEAR_Research/FourierTransformSpectrometer/Data


In [50]:
# Dump the position (in encoder count) and signal to a pickle file
with open('SimulatedData_Range_LinearTrend_zeropt.pkl', 'w') as f:
    pkl.dump(data,f)